In [1]:
import os
import mmcv
from mmcv import Config

from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test

from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

In [2]:
import  torch
print(torch.__version__)

1.6.0


In [3]:
# config file 들고오기
cfg = Config.fromfile('/opt/ml/mmsegmentation/work_dirs/upernet_swin/upernet_swin.py')
root='/opt/ml/mmsegmentation/data/images/test'

epoch = 'swin_pseudo2_Fold_4'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.work_dir = '/opt/ml/mmsegmentation/work_dirs/upernet_swin'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

In [4]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2021-11-04 04:21:34,528 - mmseg - INFO - Loaded 819 images


In [5]:
#device

In [6]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/mmsegmentation/mmseg/models/backbones/swin.py:553: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


Use load_from_local loader


In [7]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 4.7 task/s, elapsed: 176s, ETA:     0s

In [8]:
# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/segmentation/baseline_code/submission/sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/segmentation/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size

# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=False)

In [9]:
# reference : https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
input_size = 512
output_size = 256
bin_size = input_size // output_size

for image_id, predict in enumerate(output):
  temp_mask = []
  predict = predict.reshape(1, 512, 512)
  print("pred1",predict.shape)
  mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2)
  print("mask",mask.shape)
  temp_mask.append(mask)
  
  oms = np.array(temp_mask)
  oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)
  print("oms",oms.shape)
  predict = oms
  print("pred",predict.shape)

pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
mask (1, 256, 256)
oms (1, 65536)
pred (1, 65536)
pred1 (1, 512, 512)
